In [1]:
!pip install holidays

     |████████████████████████████████| 114 kB 265 kB/s eta 0:00:01
     |████████████████████████████████| 43 kB 672 kB/s eta 0:00:01
     |████████████████████████████████| 732 kB 906 kB/s eta 0:00:01
  Created wheel for holidays: filename=holidays-0.10.3-py3-none-any.whl size=115855 sha256=f734ce843d76aa5276b7cc23d0128c1efb54de95532642f08cad9951931b181d
  Stored in directory: /Users/deep.gandhi/Library/Caches/pip/wheels/d0/67/01/18308fa32fabcd0af569b9f52934b2118c9b30bcf7b4b378f8
  Created wheel for pymeeus: filename=PyMeeus-0.3.7-py3-none-any.whl size=702876 sha256=0b4126b64dd449065d5491c46149bb125a34233a758c442d7c131ccd2fa256ec
  Stored in directory: /Users/deep.gandhi/Library/Caches/pip/wheels/80/32/5f/2a67880d4ce584b9cf99146f9945e46942dfb010a9382c6ff5
Successfully built holidays pymeeus


In [21]:
import holidays
from datetime import date as date
import pandas as pd
import numpy as np

In [19]:
for date, name in sorted(holidays.IND(state='AS', years=2020).items()):
    print(date, name)

2020-01-14 Makar Sankranti / Pongal
2020-01-26 Republic Day
2020-05-01 Labour Day
2020-08-15 Independence Day
2020-10-02 Gandhi Jayanti
2020-12-25 Christmas


In [84]:
us_holidays = holidays.US()


'2014-01-01'

In [94]:
country_code = 'US'
us_holidays = holidays.CountryHoliday(country_code)
us_holidays.prov = ['CA','MX']
us_holidays.get('2020-1-2')

## Add holiday to dataframe

In [11]:
def add_holidays_to_dataframe(dataframe, date_field, country_code, province:list=None):
    country_holidays = holidays.CountryHoliday(country_code)
    country_holidays.prov = province
    #dataframe['StateHoliday'] = 0
    cols = list(dataframe.columns)
    cols.append('StateHoliday')
    #print(dataframe)
    #dates = pd.DataFrame(dataframe[date_field]).copy()
    dataframe['StateHoliday'] = 0
    #print(dates.dtypes)
    if dataframe[date_field].dtypes =='datetime64[ns]':
        dataframe[date_field] = dataframe[date_field].dt.strftime('%Y-%m-%d')
        
    dataframe.set_index(date_field, inplace=True)
    #print(dataframe)
    for i in zip(dataframe.index):
        if country_holidays.get(i[0]):
            dataframe['StateHoliday'].loc[i[0]] = 1
    dataframe.reset_index(inplace=True)
    dataframe.columns = cols
    dataframe[date_field] = pd.to_datetime(dataframe[date_field])

In [101]:
df = pd.DataFrame(pd.to_datetime(pd.Series(['20010101','20010201', '20010704','20010815']), format = '%Y%m%d'))
df.columns = ['date']
#df['date'] = df['date'].dt.strftime('%Y-%m-%d')
#df.set_index('date',inplace=True)
#df['state'] = 0
#df.loc['2001-01-01']['state']
df

,date
0,2001-01-01
1,2001-02-01
2,2001-07-04
3,2001-08-15


In [102]:
add_holidays_to_dataframe(df, 'date', 'US', ['CA'])
df

,date,StateHoliday
0,2001-01-01,1
1,2001-02-01,0
2,2001-07-04,1
3,2001-08-15,0


## Get time elapsed after a holiday or before the next one

In [100]:
def get_elapsed_after(dataframe, date_field, calculated_field, group_field='None', prefix='After'):
    hol_array = []
    res = []
    if group_field:
        last_store = 0
    last_date = dataframe[date_field].iloc[0]
    first_param = dataframe[group_field].values if not group_field else dataframe.index.values
    for s,v,d in zip(first_param,dataframe[calculated_field].values, dataframe[date_field].values):
        if group_field:
            if s != last_store:
                last_date = np.datetime64()
                last_store = s
       
        if v: 
            last_date = d
            hol_array.append(d)
        else:
            last_date = hol_array[-1]
        
        z = ((d-last_date).astype('timedelta64[D]'))/np.timedelta64(1, 'D')
        res.append(float(z))        
    dataframe[prefix+calculated_field] = res

In [103]:
get_elapsed_after(df, 'date','StateHoliday')
df

,date,StateHoliday,AfterStateHoliday
0,2001-01-01,1,0.0
1,2001-02-01,0,31.0
2,2001-07-04,1,0.0
3,2001-08-15,0,42.0
